# Computation Graphs and Back Propagation

In [103]:
import numpy as np

## Define a Set of Computation Nodes

| Name        | Fomula           | Gradients  |
|:-------------:|:------------- |:----- |
| Linear      | $y=x^T\cdot W+b$ | $\frac{\partial \mathcal{L}}{x}=W\cdot\frac{\partial \mathcal{L}}{\partial y}\\\frac{\partial \mathcal{L}}{W}=x^T\cdot\frac{\partial \mathcal{L}}{\partial y}\\\frac{\partial \mathcal{L}}{b}=\frac{\partial \mathcal{L}}{\partial y}$ |
| Sigmoid     | $y=\frac{1}{1+e^{-x}}$  | $\frac{\partial \mathcal{L}}{x}=\frac{\partial \mathcal{L}}{\partial y}(1-y)y$ |
| Softmax     | $y_j=\frac{e^{x_j}}{\sum\limits_i e^{x_i}}$ | $\frac{\partial \mathcal{L}}{x_j}=\frac{\partial \mathcal{L}}{\partial y_j}y_j-y_j\sum\limits_i \frac{\partial \mathcal{L}}{\partial y_i}y_i$ |
| CrossEntropy | $y=-\sum\limits_i p_i \log(x_i)$ | $\frac{\partial \mathcal{L}}{x_i}=-\frac{\partial \mathcal{L}}{\partial y}\frac{p_i}{x_i}$ |
| Mean  | $y=\frac{1}{N}\sum\limits_i x_i$ | $\frac{\partial \mathcal{L}}{x_i}=\frac{1}{N}\frac{\partial \mathcal{L}}{\partial y}$ |


### Linear: $y=x^T\cdot W+b, \frac{\partial \mathcal{L}}{x}=W\cdot\frac{\partial \mathcal{L}}{\partial y},\frac{\partial \mathcal{L}}{W}=x^T\cdot\frac{\partial \mathcal{L}}{\partial y},\frac{\partial \mathcal{L}}{b}=\frac{\partial \mathcal{L}}{\partial y}$
### Sigmoid: $y=\frac{1}{1+e^{-x}}, \frac{\partial \mathcal{L}}{x}=\frac{\partial \mathcal{L}}{\partial y}(1-y)y$
### Softmax: $y_j=\frac{e^{x_j}}{\sum\limits_i e^{x_i}},\frac{\partial \mathcal{L}}{x_j}=\frac{\partial \mathcal{L}}{\partial y_j}y_j-y_j\sum\limits_i \frac{\partial \mathcal{L}}{\partial y_i}y_i$
### CrossEntropy: $y=-\sum\limits_i p_i \log(x_i),\frac{\partial \mathcal{L}}{x_i}=-\frac{\partial \mathcal{L}}{\partial y}\frac{p_i}{x_i}$
### Mean: $y=\frac{1}{N}\sum\limits_i x_i,\frac{\partial \mathcal{L}}{x_i}=\frac{1}{N}\frac{\partial \mathcal{L}}{\partial y}$

In [104]:
class Linear(object):
    def __init__(self, input_shape, output_shape, mean=0, variance=0.1):
        self.parameters = [mean + variance * np.random.randn(input_shape, output_shape),
                           mean + variance * np.random.randn(output_shape)]
        self.parameters_deltas = [None, None]

    def forward(self, x, *args):
        self.x = x
        return np.matmul(x, self.parameters[0]) + self.parameters[1]

    def backward(self, delta):
        self.parameters_deltas[0] = self.x.T.dot(delta)
        self.parameters_deltas[1] = np.sum(delta, 0)
        return delta.dot(self.parameters[0].T)


class F(object):
    '''base class for functions with no parameters.'''

    def __init__(self):
        self.parameters = []
        self.parameters_deltas = []


class Sigmoid(F):
    def forward(self, x, *args):
        self.x = x
        self.y = 1.0 / (1.0 + np.exp(-x))
        return self.y

    def backward(self, delta):
        return delta * ((1 - self.y) * self.y)


class Softmax(F):
    def forward(self, x, *args):
        self.x = x
        xtmp = x - x.max(axis=-1, keepdims=True)
        exps = np.exp(xtmp)
        self.y = exps / exps.sum(axis=-1, keepdims=True)
        return self.y

    def backward(self, delta):
        return delta * self.y - self.y * (delta * self.y).sum(axis=-1, keepdims=True)


class CrossEntropy(F):
    def forward(self, x, p, *args):
        self.p = p
        self.x = x
        y = -p * np.log(np.maximum(x, 1e-15))
        return y.sum(-1)

    def backward(self, delta):
        return -delta[..., None] * 1. / np.maximum(self.x, 1e-15) * self.p


class Mean(F):
    def forward(self, x, *args):
        self.x = x
        return x.mean()

    def backward(self, delta):
        return delta * np.ones(self.x.shape) / np.prod(self.x.shape)

## Sanity Check

In [105]:
def numdiff(layer, x, var, y_delta, delta, args):
    '''numerical differenciation.'''
    var_raveled = var.ravel()

    var_delta_list = []
    for ix in range(len(var_raveled)):
        var_raveled[ix] += delta / 2.
        yplus = layer.forward(x, *args)
        var_raveled[ix] -= delta
        yminus = layer.forward(x, *args)
        var_delta = ((yplus - yminus) / delta * y_delta).sum()
        var_delta_list.append(var_delta)

        # restore changes
        var_raveled[ix] += delta / 2.
    return np.array(var_delta_list)


def sanity_check(layer, x, args=(), delta=0.01, precision=1e-3):
    '''
    perform sanity check for a layer,
    raise an assertion error if failed to pass all sanity checks.

    Args:
        layer (obj): user defined neural network layer.
        x (ndarray): input array.
        args: additional arguments for forward function.
        delta: the strength of perturbation used in numdiff.
        precision: the required precision of gradient (usually introduced by numdiff).
    '''
    y = layer.forward(x, *args)
    # y_delta is the gradient passed from the next layer, i.e. dL/dy.
    y_delta = np.random.randn(*y.shape)
    x_delta = layer.backward(y_delta)

    for var, var_delta in zip([x] + layer.parameters, [x_delta] + layer.parameters_deltas):
        var_delta_num = numdiff(layer, x, var, y_delta, delta, args)
        np.testing.assert_allclose(var_delta_num.reshape(
            *var_delta.shape), var_delta, atol=precision, rtol=precision)


In [106]:
for layer in [Linear(6, 4), Sigmoid(), Softmax(), Mean()]:
    print('checking %s' % layer.__class__.__name__)
    x = np.random.uniform(size=(5, 6))
    sanity_check(layer, x)

# we take special care of cross entropy layer here,
# it takes an additional parameter l
layer = CrossEntropy()
print('checking %s' % layer.__class__.__name__)
p = np.random.uniform(0.1, 1, [5, 6])
p = p / p.sum(axis=-1, keepdims=True)
x = np.random.uniform(0.1, 1, [5, 6])
x = x / x.sum(axis=-1, keepdims=True)
sanity_check(layer, x, args=(p,), precision=1e-1)

checking Linear
checking Sigmoid
checking Softmax
checking Mean
checking CrossEntropy


In [107]:
import subprocess
import os
import struct


def load_MNIST():
    base = "http://yann.lecun.com/exdb/mnist/"
    objects = ['t10k-images-idx3-ubyte', 't10k-labels-idx1-ubyte',
               'train-images-idx3-ubyte', 'train-labels-idx1-ubyte']
    end = ".gz"
    path = "../data/raw/"
    cmd = ["mkdir", "-p", path]
    subprocess.check_call(cmd)
    for obj in objects:
        if not os.path.isfile(path + obj):
            cmd = ["wget", base + obj + end, "-P", path]
            subprocess.check_call(cmd)
            cmd = ["gzip", "-d", path + obj + end]
            subprocess.check_call(cmd)

    def unpack(filename):
        '''unpack a single file.'''
        with open(filename, 'rb') as f:
            _, _, dims = struct.unpack('>HBB', f.read(4))
            shape = tuple(struct.unpack('>I', f.read(4))
                          [0] for d in range(dims))
            data = np.fromstring(f.read(), dtype=np.uint8).reshape(shape)
            return data

    # load objects
    data = []
    for name in objects:
        name = path + name
        data.append(unpack(name))
    labels = np.zeros([data[1].shape[0], 10])
    for i, iterm in enumerate(data[1]):
        labels[i][iterm] = 1
    data[1] = labels
    labels = np.zeros([data[3].shape[0], 10])
    for i, iterm in enumerate(data[3]):
        labels[i][iterm] = 1
    data[3] = labels
    return data

In [108]:
def random_draw(data, label, batch_size):
    perm = np.random.permutation(data.shape[0])
    data_b = data[perm[:batch_size]]
    label_b = label[perm[:batch_size]]
    return data_b.reshape([data_b.shape[0], -1]) / 255.0, label_b

In [109]:
np.random.seed(5)
batchsize = 100
learning_rate = 0.5
dim_img = 784
num_digit = 10
num_epoch = 10
train_data, train_label, test_data, test_label = load_MNIST()
num_iteration = len(train_data) // batchsize * num_epoch


def match_ratio(result, label):
    '''the match rate between result and label'''
    label_p = np.argmax(result, axis=1)
    label_t = np.argmax(label, axis=1)
    ratio = np.sum(label_p == label_t) / label_t.shape[0]
    return ratio


lossfunc = CrossEntropy()
layers = [Linear(dim_img, num_digit), Softmax(), lossfunc, Mean()]


def net_forward(x, label):
    for layer in layers:
        if layer is lossfunc:
            result = x
            x = layer.forward(x, label)
        else:
            x = layer.forward(x)
    return result, x


def net_backward(y_delta):
    for layer in layers[::-1]:
        y_delta = layer.backward(y_delta)
    return y_delta


for j in range(num_iteration):
    x, label = random_draw(train_data, train_label, batchsize)
    result, loss = net_forward(x, label)
    print("iteration = %d/%d, loss = %.4f, ratio = %.2f" %
          (j, num_iteration, loss, match_ratio(result, label)))

    net_backward(learning_rate)

    # update network parameters
    for layer in layers:
        for p, p_delta in zip(layer.parameters, layer.parameters_deltas):
            p -= p_delta  # stochastic gradient descent

x, label = random_draw(test_data, test_label, 1000)
result, loss = net_forward(x, label)
print('Test loss = %.4f, ratio = %.3f' % (loss, match_ratio(result, label)))

iteration = 0/1000, loss = 3.0850, ratio = 0.13
iteration = 1/1000, loss = 2.6967, ratio = 0.21
iteration = 2/1000, loss = 2.0212, ratio = 0.31
iteration = 3/1000, loss = 1.7928, ratio = 0.38
iteration = 4/1000, loss = 1.4996, ratio = 0.49
iteration = 5/1000, loss = 1.4240, ratio = 0.47
iteration = 6/1000, loss = 1.3433, ratio = 0.60
iteration = 7/1000, loss = 1.1618, ratio = 0.63
iteration = 8/1000, loss = 0.9574, ratio = 0.67
iteration = 9/1000, loss = 0.7831, ratio = 0.76
iteration = 10/1000, loss = 0.7865, ratio = 0.75
iteration = 11/1000, loss = 0.7722, ratio = 0.80
iteration = 12/1000, loss = 0.8128, ratio = 0.78
iteration = 13/1000, loss = 0.8264, ratio = 0.72
iteration = 14/1000, loss = 0.6225, ratio = 0.82
iteration = 15/1000, loss = 0.7114, ratio = 0.85
iteration = 16/1000, loss = 0.6563, ratio = 0.82
iteration = 17/1000, loss = 0.7819, ratio = 0.78
iteration = 18/1000, loss = 0.6334, ratio = 0.84
iteration = 19/1000, loss = 0.5958, ratio = 0.86
iteration = 20/1000, loss = 0.

iteration = 198/1000, loss = 0.3519, ratio = 0.91
iteration = 199/1000, loss = 0.2890, ratio = 0.92
iteration = 200/1000, loss = 0.2278, ratio = 0.92
iteration = 201/1000, loss = 0.3304, ratio = 0.90
iteration = 202/1000, loss = 0.3636, ratio = 0.93
iteration = 203/1000, loss = 0.4554, ratio = 0.87
iteration = 204/1000, loss = 0.2296, ratio = 0.96
iteration = 205/1000, loss = 0.2752, ratio = 0.94
iteration = 206/1000, loss = 0.4624, ratio = 0.88
iteration = 207/1000, loss = 0.2911, ratio = 0.90
iteration = 208/1000, loss = 0.3767, ratio = 0.88
iteration = 209/1000, loss = 0.2303, ratio = 0.96
iteration = 210/1000, loss = 0.3294, ratio = 0.90
iteration = 211/1000, loss = 0.2598, ratio = 0.93
iteration = 212/1000, loss = 0.4205, ratio = 0.88
iteration = 213/1000, loss = 0.2353, ratio = 0.93
iteration = 214/1000, loss = 0.2862, ratio = 0.95
iteration = 215/1000, loss = 0.3029, ratio = 0.94
iteration = 216/1000, loss = 0.2486, ratio = 0.92
iteration = 217/1000, loss = 0.2493, ratio = 0.93


iteration = 396/1000, loss = 0.3210, ratio = 0.91
iteration = 397/1000, loss = 0.4236, ratio = 0.90
iteration = 398/1000, loss = 0.1864, ratio = 0.95
iteration = 399/1000, loss = 0.1545, ratio = 0.95
iteration = 400/1000, loss = 0.3190, ratio = 0.92
iteration = 401/1000, loss = 0.3317, ratio = 0.89
iteration = 402/1000, loss = 0.1793, ratio = 0.96
iteration = 403/1000, loss = 0.2165, ratio = 0.94
iteration = 404/1000, loss = 0.2565, ratio = 0.91
iteration = 405/1000, loss = 0.4034, ratio = 0.89
iteration = 406/1000, loss = 0.3098, ratio = 0.93
iteration = 407/1000, loss = 0.4179, ratio = 0.89
iteration = 408/1000, loss = 0.3612, ratio = 0.89
iteration = 409/1000, loss = 0.2905, ratio = 0.91
iteration = 410/1000, loss = 0.2439, ratio = 0.93
iteration = 411/1000, loss = 0.2313, ratio = 0.92
iteration = 412/1000, loss = 0.2859, ratio = 0.91
iteration = 413/1000, loss = 0.2278, ratio = 0.95
iteration = 414/1000, loss = 0.2537, ratio = 0.92
iteration = 415/1000, loss = 0.2856, ratio = 0.93


iteration = 598/1000, loss = 0.2776, ratio = 0.93
iteration = 599/1000, loss = 0.2799, ratio = 0.93
iteration = 600/1000, loss = 0.1518, ratio = 0.96
iteration = 601/1000, loss = 0.2754, ratio = 0.91
iteration = 602/1000, loss = 0.2537, ratio = 0.90
iteration = 603/1000, loss = 0.1956, ratio = 0.93
iteration = 604/1000, loss = 0.1505, ratio = 0.95
iteration = 605/1000, loss = 0.1735, ratio = 0.92
iteration = 606/1000, loss = 0.1942, ratio = 0.92
iteration = 607/1000, loss = 0.1544, ratio = 0.98
iteration = 608/1000, loss = 0.2189, ratio = 0.92
iteration = 609/1000, loss = 0.1337, ratio = 0.97
iteration = 610/1000, loss = 0.2809, ratio = 0.91
iteration = 611/1000, loss = 0.2830, ratio = 0.92
iteration = 612/1000, loss = 0.0801, ratio = 0.98
iteration = 613/1000, loss = 0.2420, ratio = 0.90
iteration = 614/1000, loss = 0.1974, ratio = 0.95
iteration = 615/1000, loss = 0.3223, ratio = 0.91
iteration = 616/1000, loss = 0.2717, ratio = 0.90
iteration = 617/1000, loss = 0.1742, ratio = 0.95


iteration = 779/1000, loss = 0.2730, ratio = 0.93
iteration = 780/1000, loss = 0.3832, ratio = 0.91
iteration = 781/1000, loss = 0.2608, ratio = 0.92
iteration = 782/1000, loss = 0.2137, ratio = 0.93
iteration = 783/1000, loss = 0.1407, ratio = 0.96
iteration = 784/1000, loss = 0.2764, ratio = 0.94
iteration = 785/1000, loss = 0.1989, ratio = 0.93
iteration = 786/1000, loss = 0.1942, ratio = 0.93
iteration = 787/1000, loss = 0.3342, ratio = 0.90
iteration = 788/1000, loss = 0.2899, ratio = 0.91
iteration = 789/1000, loss = 0.2779, ratio = 0.92
iteration = 790/1000, loss = 0.3806, ratio = 0.87
iteration = 791/1000, loss = 0.3107, ratio = 0.90
iteration = 792/1000, loss = 0.2316, ratio = 0.92
iteration = 793/1000, loss = 0.2945, ratio = 0.88
iteration = 794/1000, loss = 0.1678, ratio = 0.96
iteration = 795/1000, loss = 0.1782, ratio = 0.97
iteration = 796/1000, loss = 0.2146, ratio = 0.92
iteration = 797/1000, loss = 0.1774, ratio = 0.94
iteration = 798/1000, loss = 0.2011, ratio = 0.93


iteration = 966/1000, loss = 0.2692, ratio = 0.90
iteration = 967/1000, loss = 0.1048, ratio = 0.98
iteration = 968/1000, loss = 0.2172, ratio = 0.95
iteration = 969/1000, loss = 0.1888, ratio = 0.94
iteration = 970/1000, loss = 0.2618, ratio = 0.92
iteration = 971/1000, loss = 0.4449, ratio = 0.88
iteration = 972/1000, loss = 0.2179, ratio = 0.94
iteration = 973/1000, loss = 0.2181, ratio = 0.93
iteration = 974/1000, loss = 0.2578, ratio = 0.95
iteration = 975/1000, loss = 0.1776, ratio = 0.95
iteration = 976/1000, loss = 0.3591, ratio = 0.89
iteration = 977/1000, loss = 0.2663, ratio = 0.91
iteration = 978/1000, loss = 0.2435, ratio = 0.92
iteration = 979/1000, loss = 0.2431, ratio = 0.93
iteration = 980/1000, loss = 0.3115, ratio = 0.93
iteration = 981/1000, loss = 0.1267, ratio = 0.94
iteration = 982/1000, loss = 0.1598, ratio = 0.97
iteration = 983/1000, loss = 0.1836, ratio = 0.94
iteration = 984/1000, loss = 0.3461, ratio = 0.89
iteration = 985/1000, loss = 0.1557, ratio = 0.95
